# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.  

Цель исследования:построить модель для задачи классификации, которая выберет подходящий тариф

Ход исследования:
Данные представлены в файле /datasets/users_behavior.csv. О качестве данных известно, что обзор данных не понадобится, так как предобработка уже была сделана. 

Таким образом, исследование пройдет в 5 шагов:
1. Разбить данные на выборки
2. Исследовать модели
3. Проверить модель на тестовой выборке
4. Проверить модель на адекватность
5. Общий вывод


## Откройте и изучите файл

In [32]:
import pandas as pd# импортируем библиотеку, стандартно используется имя pd

import matplotlib.pyplot as plt# импортируем библиотеку, стандартно используется имя plt

import seaborn as sns# импортируем библиотеку, стандартно используется имя sns

import numpy as np# импортируем библиотеку, стандартно используется имя np

from sklearn.tree import DecisionTreeClassifier#импортируем алгоритм  DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier#импортируем алгоритм RandomForestClassifier

from sklearn.linear_model import LogisticRegression#импортируем алгоритм LogisticRegression

from sklearn.model_selection import train_test_split# импортируйте функцию train_test_split из библиотеки sklearn

from sklearn.metrics import accuracy_score#импортируем метрическую функцию accuracy_score

from sklearn.dummy import DummyClassifier

from sklearn.model_selection import RandomizedSearchCV

In [33]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')# Чтение данных из файла
display(df.tail(2))#выводим последние строки на экран
df.shape

,calls,minutes,messages,mb_used,is_ultra
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


(3214, 5)

***Вывод***  
Итак, в представленных данных 5 столбцов.   
Данные содержат информацию о количестве звонков и сообщений, минутах разговоров, а также о количестве мегабайт использованного трафика пользователей каждого тарифа.  
Предварительно можно утверждать, что данных достаточно для построения модели для задачи классификации, которая выберет подходящий тариф.  
Поскольку в описании проекта написано: "Предобработка данных не понадобится — вы её уже сделали", было принято решение не исследовать данные.

## Разбейте данные на выборки

Разобьем данные на 3 выборки : тренировочную, валидационную и тестовую, в соотношении 3:1:1. Для этого выделим сначала тренировочную выборку, с размером train_size = 0.6. Оствшуюся выборку разобьем на 2 равных части: валидационную и тестовую.

In [34]:
df_common, df_train = train_test_split(df, train_size = 0.6, random_state=12345)# < разделим данные на обучающую>
df_test, df_valid = train_test_split(df_common, test_size = 0.5, random_state=12345)# < разделим данные на тестовую и валидационную выборки >
# < создадим переменные для признаков и целевого признака >
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)# <  >
target_valid = df_valid['is_ultra']# < >
features_test = df_valid.drop(['is_ultra'], axis=1)# <  >
target_test = df_valid['is_ultra']# < >
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(1286, 4)
(1286,)
(964, 4)
(964,)
(964, 4)
(964,)


## Исследуйте модели

In [35]:
table_decision_tree = pd.DataFrame({'model_name':[],'accuracy':[], 'type':[]})
best_result_dtc = 0
best_depth_dtc = 0
for depth_dtc in range(1, 6):
    model_dtc = DecisionTreeClassifier(random_state=12345, max_depth= depth_dtc) 
    # обучим модель с заданной глубиной дерева
    model_dtc.fit(features_train, target_train) # обучите модель
    predictions_valid_dtc = model_dtc.predict(features_valid) # получим предсказания модели
    result_dtc = accuracy_score(target_valid, predictions_valid_dtc) # посчитаем качество модели
    dictionary={'model_name':'decision_tree', 'accuracy':result_dtc, 'type':depth_dtc}
    table_decision_tree = table_decision_tree.append(dictionary, ignore_index=True)
    if result_dtc > best_result_dtc:
        best_model_dtc = model_dtc
        best_result_dtc = result_dtc
        best_depth_dtc = depth_dtc
print("Accuracy лучшей модели:", best_result_dtc, "Максимальная глубина:",  best_depth_dtc)

Accuracy лучшей модели: 0.8018672199170125 Максимальная глубина: 4


In [45]:
random_search = {'criterion': ['entropy', 'gini'],
               'max_depth': list(range(1, 11)) + [None],
               'max_features': ['auto', 'sqrt','log2', None],
               'min_samples_leaf': [4, 6, 8, 12],
               'min_samples_split': [5, 7, 10, 14],
               'n_estimators': list(range(10, 51, 10))}

clf = RandomForestClassifier()

# best_model_rfc = None
# best_result_rfc = 0
# best_est_rfc = 0
# best_depth_rfc = 0
# for est_rfc in range(10, 51, 10):
#     for depth_rfc in range (1, 11):
        #model_rfc = RandomForestClassifier(random_state=12345, n_estimators=est_rfc, max_depth=depth_rfc) # инициализируем модель RandomForestClassifier с параметром random_state=12345, n_estimators=est_rfc и max_depth=depth_rfc
model_rfc = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 80, 
                               cv = 4, verbose= 5, random_state= 12345, n_jobs = -1)   
model_rfc.fit(features_train, target_train)
predictions_valid_rfc = model_rfc.predict(features_valid)# обучим модель на тренировочной выборке
result_rfc = model_rfc.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
# if result_rfc > best_result_rfc:
# print(model_rfc.best_params_)
best_model_rfc = model_rfc# сохраним наилучшую модель
best_result_rfc = result_rfc#  сохраним наилучшее значение метрики accuracy на валидационных данных
best_est_rfc = model_rfc.best_params_['n_estimators']# сохраним наилучшую количество деревьев
best_depth_rfc = model_rfc.best_params_['max_depth']# сохраним наилучшую максимальную глубину

table_random_forest = pd.DataFrame({'model_name':['random_forest'],'accuracy':[best_result_rfc], 'type':['best']})
print("Accuracy наилучшей модели на валидационной выборке:", best_result_rfc, "Количество деревьев:", best_est_rfc, "Максимальная глубина:",  best_depth_rfc)




Fitting 4 folds for each of 80 candidates, totalling 320 fits
[CV] n_estimators=30, min_samples_split=5, min_samples_leaf=8, max_features=None, max_depth=8, criterion=gini 
[CV]  n_estimators=30, min_samples_split=5, min_samples_leaf=8, max_features=None, max_depth=8, criterion=gini, score=0.798, total=   0.1s
[CV] n_estimators=30, min_samples_split=5, min_samples_leaf=8, max_features=None, max_depth=8, criterion=gini 
[CV]  n_estimators=30, min_samples_split=5, min_samples_leaf=8, max_features=None, max_depth=8, criterion=gini, score=0.807, total=   0.1s
[CV] n_estimators=30, min_samples_split=5, min_samples_leaf=8, max_features=None, max_depth=8, criterion=gini 


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV]  n_estimators=30, min_samples_split=5, min_samples_leaf=8, max_features=None, max_depth=8, criterion=gini, score=0.757, total=   0.1s
[CV] n_estimators=30, min_samples_split=5, min_samples_leaf=8, max_features=None, max_depth=8, criterion=gini 
[CV]  n_estimators=30, min_samples_split=5, min_samples_leaf=8, max_features=None, max_depth=8, criterion=gini, score=0.769, total=   0.1s
[CV] n_estimators=30, min_samples_split=7, min_samples_leaf=8, max_features=auto, max_depth=1, criterion=gini 
[CV]  n_estimators=30, min_samples_split=7, min_samples_leaf=8, max_features=auto, max_depth=1, criterion=gini, score=0.752, total=   0.0s
[CV] n_estimators=30, min_samples_split=7, min_samples_leaf=8, max_features=auto, max_depth=1, criterion=gini 
[CV]  n_estimators=30, min_samples_split=7, min_samples_leaf=8, max_features=auto, max_depth=1, criterion=gini, score=0.773, total=   0.0s
[CV] n_estimators=30, min_samples_split=7, min_samples_leaf=8, max_features=auto, max_depth=1, criterion=gini 


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s


[CV]  n_estimators=30, min_samples_split=7, min_samples_leaf=8, max_features=auto, max_depth=1, criterion=gini, score=0.776, total=   0.0s
[CV] n_estimators=50, min_samples_split=14, min_samples_leaf=8, max_features=log2, max_depth=7, criterion=entropy 
[CV]  n_estimators=50, min_samples_split=14, min_samples_leaf=8, max_features=log2, max_depth=7, criterion=entropy, score=0.786, total=   0.1s
[CV] n_estimators=50, min_samples_split=14, min_samples_leaf=8, max_features=log2, max_depth=7, criterion=entropy 
[CV]  n_estimators=50, min_samples_split=14, min_samples_leaf=8, max_features=log2, max_depth=7, criterion=entropy, score=0.811, total=   0.1s
[CV] n_estimators=50, min_samples_split=14, min_samples_leaf=8, max_features=log2, max_depth=7, criterion=entropy 
[CV]  n_estimators=50, min_samples_split=14, min_samples_leaf=8, max_features=log2, max_depth=7, criterion=entropy, score=0.769, total=   0.1s
[CV] n_estimators=50, min_samples_split=14, min_samples_leaf=8, max_features=log2, max_

[Parallel(n_jobs=-1)]: Done 320 out of 320 | elapsed:   17.7s finished


In [37]:
model_log_r = LogisticRegression(random_state=12345)# инициализируем модель логистической регрессии с параметром random_state=12345
model_log_r.fit(features_train, target_train) # обучим модель на тренировочной выборке
predictions_log_r = model_log_r.predict(features_valid)
result_log_r = model_log_r.score(features_valid, target_valid) # получим метрику качества модели на валидационной выборке
table_log_reg = pd.DataFrame({'model_name':['logistic_regression'],'accuracy':[result_log_r], 'type':['single']})
print("Accuracy модели логистической регрессии на валидационной выборке:", result_log_r)

Accuracy модели логистической регрессии на валидационной выборке: 0.696058091286307


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


**Вывод**  
Таким образом, по результатам исследования установлено, что наибольшее по данным на валидационной выборке у модели DecisionTreeClassifier и составляет 0.8018672199170125

## Проверьте модель на тестовой выборке

In [38]:
test_predictions = model_rfc.predict(features_test) # получим предсказания модели на тестовой выборке
test_accuracy_score = accuracy_score(target_test, test_predictions)# расчитаем accuracy на тестовой выборке

print(f"Тестовая выборка: {test_accuracy_score}") 

Тестовая выборка: 0.7904564315352697


**Вывод**  
На тестовой выборке модель RandomForestClassifier 
показала accuracy = 0.80248833592535.

## (бонус) Проверьте модели на адекватность

In [39]:
model_dc = DummyClassifier(strategy='most_frequent')# инициализируем DummyClassifier
model_dc.fit(features_train, target_train) # обучим DummyClassifier на тренировочной выборке
predictions_dc = model_dc.predict(features_valid)
result_dc = model_dc.score(features_valid, target_valid) # получим метрику качества модели на валидационной выборке
table_dummy = pd.DataFrame({'model_name':['dummy_classifier'],'accuracy':[result_dc], 'type':['most_frequent']})
print("Accuracy классификатора DummyClassifier:", result_dc)

Accuracy классификатора DummyClassifier: 0.6929460580912863


In [40]:
table_sum = pd.concat([table_decision_tree, table_random_forest, table_log_reg, table_dummy])
table_sum = table_sum.reset_index()[['model_name', 'accuracy', 'type']]
table_sum.sort_values(by='accuracy')

,model_name,accuracy,type
7,dummy_classifier,0.692946,most_frequent
6,logistic_regression,0.696058,single
0,decision_tree,0.738589,1
1,decision_tree,0.775934,2
2,decision_tree,0.775934,3
4,decision_tree,0.788382,5
5,random_forest,0.790456,best
3,decision_tree,0.801867,4


***Общий вывод***  
В ходе проведения исследования было проведено изучение результатов работы 3 моделей классификации:
- DesicionTreeClassifier,
- RandomForestClassifier,
- LogisticRegression. 
По результатам проведенного исследования установлено, что наибольшую долю правильных ответов можно получить используя модель DesicionTreeClassifier с глубиной дерева = 4.

В ходе проверки моделей на адекватность, также установлено, что все модели показывают результаты лучше, чем классификатор DummyClassifier. При этом, модель LogisticRegression на валидационной выборке показывает наихудшие результаты.